# Importando as libraries

In [17]:
import os
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
import unicodedata
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
from nltk.stem import SnowballStemmer
from nltk.stem import RSLPStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split, GridSearchCV
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import glob

In [66]:
import sqlite3

In [64]:
#nltk.download('stopwords')
#nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hugos\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

# Lendo e salvando todos os arquivos de trabalhadores em um único dataframe

In [19]:
path = r'SINE/Trabalhadores' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, sep=";", encoding='iso-8859-1')
    li.append(df)

trabalhadores = pd.concat(li, axis=0, ignore_index=True)

# Criando todo o framework para o preprocessamento

In [20]:
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

In [21]:
CONTRACTION_MAP = {
    "d'aquela": "de aquela",
    "d'aquella": "de aquela",
    "d'aquellas": "de aquelas",
    "d'aquelle": "de aquele",
    "d'aquelles": "de aqueles",
    "d'aquillo": "de aquilo",
    'daquela': "de aquela",
    'daquelas': "de aquelas",
    'daquele': "de aquele",
    'daqueles': "de aqueles",
    'daquella': "de aquela",
    'daquelle': "de aquele",
    'daquelles': "de aquela",
    'daqueloutro': "de aquele outro",
    'daquillo': "de aquilo",
    'daquilo': "de aquilo",
    "n'aquella": "em aquela",
    "n'aquellas": "em aquelas",
    "n'aquelle": "em aquele",
    "n'aquelles": "em aqueles",
    "n'aquillo": "em aquilo",
    'naquela': "em aquela",
    'naquelas': "em aquelas",
    'naquele': "em aquele",
    'naqueles': "em aqueles",
    'naquella': "em aquela",
    'naquelle': "em aquele",
    'naquilo': "em aquilo",
    "d'essa": "de essa",
    "d'essas": "de essas",
    "d'esse": "de esse",
    "d'esses": "de essa",
    "d'esta": "de esta",
    "d'estas": "de estas",
    "d'este": "de este",
    "d'estes": "de estes",
    'dessa': "de essa",
    'dessas': "de essas",
    'desses': "de esses",
    'desta': "de esta",
    'destas': "de estas",
    'deste': "de este",
    'destes': "de estes",
    "n'essa": "em essa",
    "n'essas": "em essas",
    "n'esse": "em esse",
    "n'esses": "em esses",
    "n'esta": "em esta",
    "n'estas": "em estas",
    "n'este": "em este",
    "n'estes": "em estes",
    'nessa': "em essa",
    'nessas': "em essas",
    'nesse': "em esse",
    'nesses': "em esses",
    'nesta': "em esta",
    'nestas': "em estas",
    'neste': "em este",
    'nestes': "em estes",
    "d'isso": "de isso",
    "d'isto": "de isto",
    'disso': "de isso",
    'disto': "de isto",
    "n'isso": "em isso",
    "n'isto": "em isto",
    'nisso': "em isso",
    'nisto': "em isto"
}

In [22]:
def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), flags=re.IGNORECASE|re.DOTALL)
    
    def expand_match(contraction):
        match = contraction.group(0)
        expanded_contraction = contraction_mapping.get(match) if contraction_mapping.get(match) else contraction_mapping.get(match.lower())
        expanded_contraction = expanded_contraction[0:]
        return expanded_contraction
    
    expanded_text = contractions_pattern.sub(expand_match, text)
    return expanded_text

In [23]:
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s\-]' if not remove_digits else r'[^a-zA-z\s\-]'
    text = re.sub(pattern, '', text)
    return text

In [24]:
def caseconversion(text, case_type = 'lower'):
    if case_type == 'lower':
        text = text.lower()
    elif case_type == 'upper':
        text = text.upper()
    elif case_type == 'title':
        text = text.title()
    return text

In [25]:
def simple_stemmer(text, stemmer='RSLP'):
    ps = RSLPStemmer()
    if stemmer=='Snowball':
        ps = SnowballStemmer("portuguese")
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

In [183]:
stopword_list = nltk.corpus.stopwords.words('portuguese')
new_stopwords = ['ate', 'até', 'ano', 'vai', 'sera', 'será']
stopword_list.extend(new_stopwords)

In [82]:
def remove_stopwords(text, is_lower_case=False):
    tokens = nltk.word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

In [27]:
def preprocess_corpus(corpus, contraction_expansion=True, accented_char_removal=True, 
                     text_lower_case=True, stem_or_lemma='stem', special_char_removal=True, stopword_removal=True, 
                     remove_digits=False):
    preprocessed_corpus = []
    # preprocess each document in the corpus
    for doc in tqdm(corpus):
        # remove accented characters
        if accented_char_removal:
            doc = remove_accented_chars(doc)
        # expand contractions
        if contraction_expansion:
            doc = expand_contractions(doc)
        # lowercase the text
        if text_lower_case:
            doc = doc.lower()
        # remove extra newlines
            doc = re.sub(r'[\r|\n|\r\n]+', ' ',doc)
        # lemmatize or stem text
#         if stem_or_lemma == 'stem':
#             doc = simple_stemmer(doc, stemmer='RSLP')
#         elif stem_or_lemma == 'lemma':
#             doc = lemmatize_text(doc)
#         else doc = doc
        # remove special characters and\or digits
        if special_char_removal:
            # insert spaces between special characters to isolate them
            special_char_pattern = re.compile(r'([{.(-)!}])')
            doc = special_char_pattern.sub(" \\1 ", doc)
            doc = remove_special_characters(doc, remove_digits=remove_digits)
        # remove extra whitespace
        doc = re.sub(' +', ' ', doc)
        # remove stopwords
        if stopword_removal:
            doc = remove_stopwords(doc, is_lower_case=text_lower_case)
        preprocessed_corpus.append(doc)
    return preprocessed_corpus

# Criando um dataframe apenas com trabalhadores que cadastraram cursos profissionalizantes

In [28]:
trab_com_cursos = trabalhadores[trabalhadores.CURSOS_PROFISSIONALIZANTES.notnull()]

# Explodindo o campo PRETENSOES em linhas diferentes linhas, caso o trabalhador possua mais de um CBO cadastrado.

In [29]:
trab_com_cursos.PRETENSOES = trab_com_cursos.PRETENSOES.str.split('|')
trab_com_cursos = trab_com_cursos.explode('PRETENSOES')

# Removendo os registros com o campo PRETENSOES nulo

In [30]:
trab_com_cursos = trab_com_cursos[trab_com_cursos.PRETENSOES.notnull()]

# Criando um campo com Código da Ocupação a partir do campo PRETENSOES

In [31]:
trab_com_cursos['cod_ocupacao'] = trab_com_cursos.PRETENSOES.str.split('-').apply(lambda x:x[0])

In [38]:
trab_com_cursos['pretensao'] = trab_com_cursos.PRETENSOES.str.split('(').apply(lambda x:x[0])
trab_com_cursos['pretensao'] = trab_com_cursos.pretensao.str.split('-').apply(lambda x:x[1])

In [89]:
trabalhadores.PRETENSOES = trabalhadores.PRETENSOES.astype(str)

In [91]:
trabalhadores['pretensao'] = trabalhadores.PRETENSOES.str.split('(').apply(lambda x:x[0])
trabalhadores['pretensao'] = trabalhadores.pretensao.str.split('-').apply(lambda x:x[1] if len(x)>1 else x[0])

In [86]:
trab_com_cursos.head(3)

,NACIONALIDADE,DEFICIENCIAS,BAIRRO,CEP,CODIGO_MUNICIPIO_IBGE,NOME_MUNICIPIO,UF,ESCOLARIDADE,ESTUDANTE,CURSOS_PROFISSIONALIZANTES,...,IDIOMAS,HABILITACAO,VEICULOS,DISP_VIAJAR,DISP_DORMIR_EMP,DISP_AUSENTAR_DOMIC,PRETENSOES,MUNICIPIOS_INTERESSE,cod_ocupacao,pretensao
8,BRASILEIRA,NaN,JARDIM MARIA DE FATIMA,1.32205e+07,355650,VARZEA PAULISTA,SP,Médio Completo,S,auxliar administrativo,...,NaN,Nenhum,N,N,N,N,"374110-Auxiliar técnico de montagem(N,19,0,Ind...",NaN,374110,Auxiliar técnico de montagem
8,BRASILEIRA,NaN,JARDIM MARIA DE FATIMA,1.32205e+07,355650,VARZEA PAULISTA,SP,Médio Completo,S,auxliar administrativo,...,NaN,Nenhum,N,N,N,N,"514320-Auxiliar de limpeza(N,0,13,Indiferente)",NaN,514320,Auxiliar de limpeza
8,BRASILEIRA,NaN,JARDIM MARIA DE FATIMA,1.32205e+07,355650,VARZEA PAULISTA,SP,Médio Completo,S,auxliar administrativo,...,NaN,Nenhum,N,N,N,N,521110-Vendedor - no comércio de mercadorias(N...,NaN,521110,Vendedor


In [92]:
trabalhadores.head(3)

,NACIONALIDADE,DEFICIENCIAS,BAIRRO,CEP,CODIGO_MUNICIPIO_IBGE,NOME_MUNICIPIO,UF,ESCOLARIDADE,ESTUDANTE,CURSOS_PROFISSIONALIZANTES,...,POS_GRADUACOES,IDIOMAS,HABILITACAO,VEICULOS,DISP_VIAJAR,DISP_DORMIR_EMP,DISP_AUSENTAR_DOMIC,PRETENSOES,MUNICIPIOS_INTERESSE,pretensao
0,BRASILEIRA,NaN,VILA UNIDOS,1.22145e+07,354990,SAO JOSE DOS CAMPOS,SP,Superior Incompleto,S,NaN,...,NaN,NaN,AB,N,N,N,N,"521110-Vendedor de comércio varejista(N,54,0,I...",NaN,Vendedor de comércio varejista
1,BRASILEIRA,NaN,MONTANHAO,9.78414e+06,354870,SAO BERNARDO DO CAMPO,SP,Médio Completo,N,NaN,...,NaN,NaN,Nenhum,N,N,N,N,"513505-Ajudante de cozinha(N,0,0,Indiferente)|...",NaN,Ajudante de cozinha
2,BRASILEIRA,NaN,JARDIM SANTA TEREZINHA (ZON,8.43002e+06,355030,SAO PAULO,SP,Médio Completo,N,NaN,...,NaN,NaN,Nenhum,N,N,N,N,"411005-Auxiliar administrativo(N,0,0,Indiferen...",NaN,Auxiliar administrativo


In [41]:
trab_com_cursos.to_pickle("./trabalhadores.pkl")

In [34]:
# trab_com_cursos = pd.read_pickle("./trabalhadores.pkl")

# Extraindo as 10 principais ocupações procuradas (com cursos)

In [214]:
top_10_ocup_com_cursos = list(trab_com_cursos.pretensao.value_counts().index[:10])

In [215]:
top_10_ocup_com_cursos

['Auxiliar de linha de produção',
 'Auxiliar de limpeza',
 'Operador de caixa',
 'Recepcionista atendente',
 'Auxiliar administrativo',
 'Repositor de mercadorias',
 'Porteiro',
 'Assistente administrativo',
 'Atendente balconista',
 'Atendente de lanchonete']

In [61]:
print(f'As ocupações mais procuradas pelos trabalhadores são: {top_10_ocup_com_cursos}')

As ocupações mais procuradas pelos trabalhadores são: ['Auxiliar de linha de produção', 'Auxiliar de limpeza', 'Operador de caixa', 'Recepcionista atendente', 'Auxiliar administrativo', 'Repositor de mercadorias', 'Porteiro', 'Assistente administrativo', 'Atendente balconista', 'Atendente de lanchonete']


# Criando o corpus apenas para a principal ocupação

In [62]:
corpus = trab_com_cursos.loc[trab_com_cursos.pretensao == top_10_ocup_com_cursos[0],'CURSOS_PROFISSIONALIZANTES'].copy()

In [65]:
corpus = preprocess_corpus(corpus)

100%|████████████████████████████████████████████████████████████████████████| 203989/203989 [00:59<00:00, 3442.18it/s]


In [67]:
def get_top_n_words(corpus, n=10, n_gram=2):
    """
    List the top n words in a vocabulary according to occurrence in a text corpus.
    
    get_top_n_words(["I love Python", "Python is a language programming", "Hello world", "I love the world"]) -> 
    [('python', 2),
     ('world', 2),
     ('love', 2),
     ('hello', 1),
     ('is', 1),
     ('programming', 1),
     ('the', 1),
     ('language', 1)]
    """
    vec = CountVectorizer(ngram_range=(n_gram,n_gram)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [70]:
t = get_top_n_words(corpus)

In [73]:
t

[('informatica basica', 50907),
 ('operador empilhadeira', 10800),
 ('informatica basico', 9485),
 ('ensino medio', 7290),
 ('cursou ate', 5626),
 ('ano ensino', 5544),
 ('auxiliar administrativo', 4000),
 ('controlador acesso', 3792),
 ('seguranca trabalho', 3440),
 ('desenho tecnico', 2999)]

# Agora, para as 10 ocupações com maior procura, vamos rodar e gravar os resultados. Quais são as características dos candidatos à mesma ocupação?

In [83]:
top_caracteristicas = {}

In [84]:
for job in top_10_ocup_com_cursos:
    corpus = trab_com_cursos.loc[trab_com_cursos.pretensao == job,'CURSOS_PROFISSIONALIZANTES'].copy()
    corpus = preprocess_corpus(corpus)
    top_caracteristicas[job] = get_top_n_words(corpus)

100%|██████████████████████████████████████████████████████████████████████████| 56258/56258 [00:11<00:00, 5081.87it/s]


In [85]:
top_caracteristicas

{'Auxiliar de linha de produção': [('informatica basica', 50907),
  ('operador empilhadeira', 10800),
  ('informatica basico', 9485),
  ('ensino medio', 7290),
  ('auxiliar administrativo', 4000),
  ('controlador acesso', 3792),
  ('seguranca trabalho', 3440),
  ('desenho tecnico', 2999),
  ('leitura interpretacao', 2945),
  ('interpretacao desenho', 2899)],
 'Auxiliar de limpeza': [('informatica basica', 26325),
  ('ensino medio', 6334),
  ('informatica basico', 5456),
  ('ensino fundamental', 4820),
  ('controlador acesso', 3941),
  ('serie ensino', 3756),
  ('parou serie', 2720),
  ('cuidador idosos', 2687),
  ('cursou serie', 2563),
  ('5a serie', 2025)],
 'Operador de caixa': [('informatica basica', 35611),
  ('informatica basico', 7122),
  ('auxiliar administrativo', 4025),
  ('ensino medio', 3772),
  ('atendimento cliente', 2850),
  ('operador caixa', 2277),
  ('tecnico administracao', 2268),
  ('gestao empresarial', 2009),
  ('assistente administrativo', 1964),
  ('recursos hum

# Quais são as demais características dos candidatos à mesma função?

In [105]:
top_outras_caracteristicas = {}

Vamos obter as 10 ocupações com maior procura, com ou sem cursos.

In [244]:
top_10_ocup = list(trabalhadores.pretensao.value_counts().index[:10])

In [250]:
for job in tqdm(top_10_ocup):
    temp_trab = trabalhadores[trabalhadores.pretensao == job]
    top_outras_caracteristicas[job] = {'ESCOLARIDADE': (temp_trab.ESCOLARIDADE.value_counts(dropna=False)[:5]/temp_trab.shape[0]).to_dict(),
                                       'POS_GRADUACOES': (temp_trab.POS_GRADUACOES.isna().value_counts()/temp_trab.shape[0]).to_dict(),
                                       'IDIOMAS': (temp_trab.IDIOMAS.value_counts(dropna=False)[:5]/temp_trab.shape[0]).to_dict(),
                                       'HABILITACAO': (temp_trab.HABILITACAO.value_counts(dropna=False)[:5]/temp_trab.shape[0]).to_dict(),
                                       'VEICULOS': (temp_trab.VEICULOS.value_counts(dropna=False)[:5]/temp_trab.shape[0]).to_dict(),
                                       'DISP_VIAJAR': (temp_trab.DISP_VIAJAR.value_counts(dropna=False)[:5]/temp_trab.shape[0]).to_dict(),
                                       'DISP_DORMIR_EMP': (temp_trab.DISP_DORMIR_EMP.value_counts(dropna=False)[:5]/temp_trab.shape[0]).to_dict(),
                                       'DISP_AUSENTAR_DOMIC': (temp_trab.DISP_AUSENTAR_DOMIC.value_counts(dropna=False)[:5]/temp_trab.shape[0]).to_dict()
                                       }

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:10<00:00,  1.07s/it]


In [251]:
top_outras_caracteristicas

{'Operador de caixa': {'ESCOLARIDADE': {'Médio Completo': 0.7849883047449321,
   'Médio Incompleto': 0.10326631766540433,
   'Fundamental Completo': 0.042228224548897304,
   'Superior Incompleto': 0.03261444642459345,
   'Superior Completo': 0.015238638895076855},
  'POS_GRADUACOES': {True: 0.9996936957006015, False: 0.00030630429939852976},
  'IDIOMAS': {nan: 0.9700309088883938,
   'Inglês Básico': 0.014528569837380262,
   'Espanhol Básico': 0.002937736689685899,
   'Inglês Intermediário': 0.0027776230786366674,
   'Inglês Avançado|Espanhol Avançado|Francês Avançado|Italiano Avançado|Alemão Avançado': 0.0027567386945867678},
  'HABILITACAO': {'Nenhum': 0.8715958453998663,
   'AB': 0.07030379817331255,
   'B': 0.04795054577856984,
   'A': 0.004796446870126977,
   'AD': 0.00201882379149031},
  'VEICULOS': {'N': 0.9756905769659167, 'S': 0.024309423034083316},
  'DISP_VIAJAR': {'N': 0.8297574626865671, 'S': 0.17024253731343283},
  'DISP_DORMIR_EMP': {'N': 0.8571786589440855, 'S': 0.142821

In [125]:
for key, value in top_outras_caracteristicas.items():
    print('Ocupação:', key)
    for k, v in top_outras_caracteristicas[key].items():
        print('\n')
        print('Característica:', k)
        print('Distribuição percentual:\n')
        print(v)

Ocupação: Operador de caixa


Característica: ESCOLARIDADE
Distribuição percentual:

Médio Completo          0.784988
Médio Incompleto        0.103266
Fundamental Completo    0.042228
Superior Incompleto     0.032614
Superior Completo       0.015239
Name: ESCOLARIDADE, dtype: float64


Característica: POS_GRADUACOES
Distribuição percentual:

True     0.999694
False    0.000306
Name: POS_GRADUACOES, dtype: float64


Característica: IDIOMAS
Distribuição percentual:

NaN                                                                                     0.970031
Inglês Básico                                                                           0.014529
Espanhol Básico                                                                         0.002938
Inglês Intermediário                                                                    0.002778
Inglês Avançado|Espanhol Avançado|Francês Avançado|Italiano Avançado|Alemão Avançado    0.002757
Name: IDIOMAS, dtype: float64


Característica

# Lendo e salvando todos os arquivos de vagas em um único dataframe

In [143]:
path = r'SINE/Vagas' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in tqdm(all_files):
    df = pd.read_csv(filename, sep=";", encoding='iso-8859-1', usecols = list(range(12)))
    li.append(df)

vagas = pd.concat(li, axis=0, ignore_index=True)

100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [00:00<00:00, 92.71it/s]


# Eliminando todos os registros que possuem o campo OBSERVACOES_OCUPACAO nulo.

In [144]:
vagas_com_observacoes = vagas[vagas.OBSERVACOES_OCUPACAO.notnull()]

In [145]:
vagas_com_observacoes.shape

(1759, 12)

# Criando o corpus apenas para vagas da principal ocupação

In [202]:
todas_as_vagas = list(vagas_com_observacoes.TITULO_OCUPACAO.value_counts().index)

In [193]:
top_exigencias = {}

In [203]:
for job in todas_as_vagas:
    corpus_vagas = vagas_com_observacoes.loc[vagas_com_observacoes.TITULO_OCUPACAO == job,'OBSERVACOES_OCUPACAO'].copy()
    corpus_vagas = preprocess_corpus(corpus_vagas)
    try:
        top_exigencias[job] = get_top_n_words(corpus_vagas)
    except:
        top_exigencias[job] = 'Não existe vaga para essa ocupação.'

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 716.14it/s]


100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 376.04it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 1002.94it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1002.46it/s]


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 501.41it/s]


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 334.02it/s]


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 501.29it/s]


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 501.23it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1002.22it/s]


In [204]:
top_exigencias

{'Operador de caixa': [('fechamento caixa', 5),
  ('ensino medio', 4),
  ('recebimento valores', 3),
  ('atendimento cliente', 3),
  ('abertura fechamento', 3),
  ('disponibilidade horarios', 3),
  ('valores fechamento', 2),
  ('caixa emissao', 2),
  ('emissao notas', 2),
  ('notas fiscais', 2)],
 'Auxiliar de escritório': [('trabalho ser', 1),
  ('ser executado', 1),
  ('executado preparacao', 1),
  ('preparacao documentos', 1),
  ('documentos digitalizacao', 1),
  ('digitalizacao geracao', 1),
  ('geracao imagens', 1),
  ('11 auxiliar', 1),
  ('auxiliar atividades', 1),
  ('atividades escritorio', 1)],
 'Auxiliar administrativo': [('medio completo', 7),
  ('ensino medio', 6),
  ('auxiliar administrativo', 5),
  ('pacote office', 5),
  ('rotinas administrativas', 4),
  ('enviar curriculo', 4),
  ('segunda sexta', 4),
  ('vaga exclusiva', 3),
  ('exclusiva pcd', 3),
  ('horario trabalho', 3)],
 'Vendedor de comércio varejista': [('vendedor loja', 1),
  ('loja material', 1),
  ('materia

# Preparando os resultados para salvar em BD

In [210]:
maisProcuradas = pd.DataFrame(top_10_ocup, columns=['pretensao'])

In [216]:
maisProcuradasComCursos = pd.DataFrame(top_10_ocup_com_cursos, columns=['pretensao'])

In [277]:
def flat_caracteristicas(data):
    flat = [[(k, v) for v in vs] for k, vs in data.items()]
    flat2 = []
    for i in flat:
        for j in i:
            try:
                flat2.append([j[0], j[1][0], j[1][1]])
            except:
                flat2.append([j[0], '', ''])
    return flat2

In [238]:
 flatten_top_caracteristicas = flat_caracteristicas(top_caracteristicas)

In [241]:
topCaracteristicasConcorrentes = pd.DataFrame.from_records(flatten_top_caracteristicas, columns=['pretensao', 'caracteristica', 'ocorrencias'])

In [243]:
topCaracteristicasConcorrentes

,pretensao,caracteristica,ocorrencias
0,Auxiliar de linha de produção,informatica basica,50907
1,Auxiliar de linha de produção,operador empilhadeira,10800
2,Auxiliar de linha de produção,informatica basico,9485
3,Auxiliar de linha de produção,ensino medio,7290
4,Auxiliar de linha de produção,auxiliar administrativo,4000
...,...,...,...
95,Atendente de lanchonete,gestao empresarial,854
96,Atendente de lanchonete,controlador acesso,817
97,Atendente de lanchonete,atendimento cliente,771
98,Atendente de lanchonete,ens medio,695


In [266]:
flatten_top_outras_caracteristicas = []
for ks, vs in top_outras_caracteristicas.items():
    for kk, vv in vs.items():
        for k, v in vv.items():
            flatten_top_outras_caracteristicas.append(list((ks, kk, k, round(v,3))))

In [268]:
topOutrasCaracteristicasConcorrentes = pd.DataFrame.from_records(flatten_top_outras_caracteristicas, columns=['pretensao', 'caracteristica', 'valor','proporcao'])

In [269]:
topOutrasCaracteristicasConcorrentes

,pretensao,caracteristica,valor,proporcao
0,Operador de caixa,ESCOLARIDADE,Médio Completo,0.785
1,Operador de caixa,ESCOLARIDADE,Médio Incompleto,0.103
2,Operador de caixa,ESCOLARIDADE,Fundamental Completo,0.042
3,Operador de caixa,ESCOLARIDADE,Superior Incompleto,0.033
4,Operador de caixa,ESCOLARIDADE,Superior Completo,0.015
...,...,...,...,...
245,Estoquista,DISP_VIAJAR,S,0.239
246,Estoquista,DISP_DORMIR_EMP,N,0.783
247,Estoquista,DISP_DORMIR_EMP,S,0.217
248,Estoquista,DISP_AUSENTAR_DOMIC,N,0.791


In [278]:
flatten_top_exigencias = flat_caracteristicas(top_exigencias)

In [280]:
topExigenciasVagas = pd.DataFrame.from_records(flatten_top_caracteristicas, columns=['pretensao', 'caracteristica', 'ocorrencias'])

# Salvando os resultados no banco SQLite

In [77]:
conn = sqlite3.connect('coronathon_db.sqlite')

In [78]:
cur = conn.cursor()

In [79]:
cur.execute('CREATE TABLE pre_maisprocuradas (pretensao VARCHAR)')
conn.commit()

In [282]:
maisProcuradas.to_sql(name='pre_maisprocuradas', con=conn, if_exists='replace')

In [283]:
maisProcuradasComCursos.to_sql(name='pre_maisProcuradasComCursos', con=conn, if_exists='replace')

In [284]:
topCaracteristicasConcorrentes.to_sql(name='pre_topCaracteristicasConcorrentes', con=conn, if_exists='replace')

In [285]:
topOutrasCaracteristicasConcorrentes.to_sql(name='pre_topOutrasCaracteristicasConcorrentes', con=conn, if_exists='replace')

In [286]:
topExigenciasVagas.to_sql(name='pre_topExigenciasVagas', con=conn, if_exists='replace')

In [293]:
cur.execute("SELECT name, sql FROM sqlite_master WHERE type='table' ORDER BY name;")
print(cur.fetchall())

[('pre_maisProcuradasComCursos', 'CREATE TABLE "pre_maisProcuradasComCursos" (\n"index" INTEGER,\n  "pretensao" TEXT\n)'), ('pre_maisprocuradas', 'CREATE TABLE "pre_maisprocuradas" (\n"index" INTEGER,\n  "pretensao" TEXT\n)'), ('pre_topCaracteristicasConcorrentes', 'CREATE TABLE "pre_topCaracteristicasConcorrentes" (\n"index" INTEGER,\n  "pretensao" TEXT,\n  "caracteristica" TEXT,\n  "ocorrencias" INTEGER\n)'), ('pre_topExigenciasVagas', 'CREATE TABLE "pre_topExigenciasVagas" (\n"index" INTEGER,\n  "pretensao" TEXT,\n  "caracteristica" TEXT,\n  "ocorrencias" INTEGER\n)'), ('pre_topOutrasCaracteristicasConcorrentes', 'CREATE TABLE "pre_topOutrasCaracteristicasConcorrentes" (\n"index" INTEGER,\n  "pretensao" TEXT,\n  "caracteristica" TEXT,\n  "valor" TEXT,\n  "proporcao" REAL\n)')]


In [297]:
def to_csv():
    db = sqlite3.connect('coronathon_db.sqlite')
    cursor = db.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    for table_name in tables:
        table_name = table_name[0]
        table = pd.read_sql_query("SELECT * from %s" % table_name, db)
        table.to_csv(table_name + '.csv', index_label='index')
    cursor.close()
    db.close()

In [298]:
to_csv()